In [33]:
import sqlite3

class AtterbergLimitsModule:
    def __init__(self, db_connection):
        self.db_connection_path = db_connection
        self.conn = None

    def __enter__(self):
        self.conn = sqlite3.connect(self.db_connection_path)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        if self.conn:
            self.conn.close()
            
    def validate_atterberg_data(self, data):
        """ Validates input data for Atterberg limits testing. """
        if not data.get('LiquidLimitNotObtained'):
            required_fields = ['LiquidLimitTareWeight', 'LiquidLimitWetWeight', 'LiquidLimitDryWeight', 'LiquidLimitNumberOfBlows']
            for field in required_fields:
                if field not in data or data[field] is None:
                    return False, f"{field} is required for Liquid Limit."
            if data['LiquidLimitWetWeight'] <= data['LiquidLimitDryWeight']:
                return False, "Liquid Limit Wet Weight must be greater than Dry Weight."
        
        if not data.get('PlasticLimitNotObtained'):
            required_fields = ['PlasticLimitTareWeight', 'PlasticLimitWetWeight', 'PlasticLimitDryWeight']
            for field in required_fields:
                if field not in data or data[field] is None:
                    return False, f"{field} is required for Plastic Limit."
            if data['PlasticLimitWetWeight'] <= data['PlasticLimitDryWeight']:
                return False, "Plastic Limit Wet Weight must be greater than Dry Weight."
        
        return True, ""

    def calculate_percent_moisture(self, tare_weight, wet_weight, dry_weight):
        """ Calculates the percent moisture of a soil sample. """
        water_weight = wet_weight - dry_weight
        dry_soil_weight = dry_weight - tare_weight
        return round((water_weight / dry_soil_weight) * 100, 2)

    def calculate_liquid_limit(self, percent_moisture, blows):
        """ Calculates the Liquid Limit based on percent moisture and number of blows. """
        if blows is None or blows <= 0:
            return None
        correction_factor = (blows / 25) ** 0.121
        return round(percent_moisture * correction_factor, 2)

    def calculate_plastic_limit(self, percent_moisture):
        """ Calculates the Plastic Limit as the moisture content. """
        return percent_moisture

    def calculate_plasticity_index(self, liquid_limit, plastic_limit):
        """ Calculates the Plasticity Index. """
        if liquid_limit is None or plastic_limit is None:
            return None
        return round(liquid_limit - plastic_limit, 2)

    def fetch_data(self, sample_id):
        """Fetches the Atterberg limits data for a given SampleID from the database."""
        cursor = self.conn.cursor()
        query = """
        SELECT SampleID, LiquidLimitTareWeight, LiquidLimitWeight,
               LiquidLimitDryWeight, LiquidLimitNumberOfBlows, LiquidLimitNotObtained,
               PlasticLimitTareWeight, PlasticLimitWetWeight, PlasticLimitDryWeight,
               PlasticLimitNotObtained
        FROM AtterbergLimitResults
        WHERE SampleID = ?
        """
        cursor.execute(query, (sample_id,))
        row = cursor.fetchone()
        
        if row:
            # Map the row data to a dictionary with appropriate keys
            data = {
                'SampleID': row[0],
                'LiquidLimitTareWeight': row[1],
                'LiquidLimitWetWeight': row[2],
                'LiquidLimitDryWeight': row[3],
                'LiquidLimitNumberOfBlows': row[4],
                'LiquidLimitNotObtained': row[5],
                'PlasticLimitTareWeight': row[6],
                'PlasticLimitWetWeight': row[7],
                'PlasticLimitDryWeight': row[8],
                'PlasticLimitNotObtained': row[9]
                # 'Remarks': row[10]
            }
            return data
        else:
            return None

    def store_atterberg_results(self, data):
        """ Stores the Atterberg Limits results in the SQLite database. """
        cursor = self.conn.cursor()
        query = """
        INSERT INTO AtterbergLimitResults (SampleID, LiquidLimit, PlasticLimit, PlasticityIndex)
        VALUES (?, ?, ?, ?)
        """
        values = (
            data.get('SampleID'),
            data.get('LiquidLimit'),
            data.get('PlasticLimit'),
            data.get('PlasticityIndex'),
            # data.get('Remarks')
        )
        cursor.execute(query, values)
        self.conn.commit()

    def process_atterberg_data(self, data):
        """ Main function to process Atterberg Limits data. """
        # Step 1: Validate input data
        is_valid, error_message = self.validate_atterberg_data(data)
        if not is_valid:
            return {"success": False, "message": error_message}

        # Step 2: Perform calculations
        liquid_limit = None
        plastic_limit = None
        plasticity_index = None

        # Calculate Liquid Limit if not marked as 'Not Obtained'
        if not data.get('LiquidLimitNotObtained'):
            percent_moisture_liquid = self.calculate_percent_moisture(
                data['LiquidLimitTareWeight'],
                data['LiquidLimitWetWeight'],
                data['LiquidLimitDryWeight']
            )
            liquid_limit = self.calculate_liquid_limit(percent_moisture_liquid, data['LiquidLimitNumberOfBlows'])

        # Calculate Plastic Limit if not marked as 'Not Obtained'
        if not data.get('PlasticLimitNotObtained'):
            percent_moisture_plastic = self.calculate_percent_moisture(
                data['PlasticLimitTareWeight'],
                data['PlasticLimitWetWeight'],
                data['PlasticLimitDryWeight']
            )
            plastic_limit = self.calculate_plastic_limit(percent_moisture_plastic)

        # Calculate Plasticity Index if both limits are obtained
        if liquid_limit is not None and plastic_limit is not None:
            plasticity_index = self.calculate_plasticity_index(liquid_limit, plastic_limit)

        # Step 3: Store results in the database
        try:
            self.store_atterberg_results({
                'SampleID': data.get('SampleID'),
                'LiquidLimit': liquid_limit,
                'PlasticLimit': plastic_limit,
                'PlasticityIndex': plasticity_index
                # 'Remarks': data.get('Remarks')
            })
            return {"success": True, "message": "Data processed successfully"}
        except Exception as e:
            return {"success": False, "message": str(e)}


In [39]:
db_path = 'Soil_framework.sqlite'
sample_id = 80  # Use the SampleID you want to process

with AtterbergLimitsModule(db_path) as atterberg_module:
    data = atterberg_module.fetch_data(sample_id)
    if data:
        result = atterberg_module.process_atterberg_data(data)
        print(result)
    else:
        print(f"No data found for SampleID {sample_id}")


{'success': True, 'message': 'Data processed successfully'}
